In [ ]:
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
import numpy as np
import os
import pandas as pd

In [ ]:
data = pd.read_csv("cars.csv")
data = data[["car_brand","image"]]

In [ ]:
os.mkdir("cars")

In [ ]:
os.chdir("cars")

In [ ]:
for i,row in enumerate(data.values):
  # if row[0] not in os.listdir():
  #   os.mkdir(row[0])
  os.system(f"wget {row[1]} -O {row[0]}/{row[0]}{i}.{row[1].split('.')[-1]} ")

In [ ]:
classes = set()
for row in data.values:
  classes.add(row[0])
classes = list(classes)

In [21]:
os.chdir("train")
for brand in classes:
  os.mkdir(brand)

In [28]:
!pwd

/content


In [29]:
train_batches = ImageDataGenerator(preprocessing_function=mobilenet_v2.preprocess_input).flow_from_directory("./cars",classes=classes,target_size=(224,224))

Found 605 images belonging to 24 classes.


In [31]:
mobilenet_model = mobilenet_v2.MobileNetV2()

14540800/14536120 [==============================] - 0s 0us/step


In [33]:
mobilenet_model.summary()


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [40]:
x = mobilenet_model.layers[-2].output
prediction_layer = Dense(len(classes),activation="softmax")(x)

model = Model(mobilenet_model.input,prediction_layer)

In [41]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [42]:
model.fit(train_batches,epochs=9)

Epoch 1/9
19/19 [==============================] - 7s 203ms/step - loss: 0.5841 - accuracy: 0.8512
Epoch 2/9
19/19 [==============================] - 4s 206ms/step - loss: 0.1644 - accuracy: 0.9488
Epoch 3/9
19/19 [==============================] - 4s 206ms/step - loss: 0.1457 - accuracy: 0.9587
Epoch 4/9
19/19 [==============================] - 4s 208ms/step - loss: 0.0750 - accuracy: 0.9802
Epoch 5/9
19/19 [==============================] - 4s 208ms/step - loss: 0.0523 - accuracy: 0.9868
Epoch 6/9
19/19 [==============================] - 4s 205ms/step - loss: 0.0329 - accuracy: 0.9917
Epoch 7/9
19/19 [==============================] - 4s 203ms/step - loss: 0.0279 - accuracy: 0.9851
Epoch 8/9
19/19 [==============================] - 4s 206ms/step - loss: 0.0494 - accuracy: 0.9835
Epoch 9/9
19/19 [==============================] - 4s 207ms/step - loss: 0.0754 - accuracy: 0.9802


In [43]:
model.save("classification_model.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [68]:
image = load_img("./cars/OPEL/OPEL89.jpg",target_size=(224,224))
image = img_to_array(image)
# image = mobilenet_v2.preprocess_input(image)
image = np.expand_dims(image,0)
image.shape

(1, 224, 224, 3)

In [69]:
prediction = model.predict(image)
prediction = classes[np.argmax(prediction)]
prediction

'SEAT'